## Hitters Veri Seti ile Maaş Tahmin Modelleri Kıyaslama

#### AMAÇ

Bu çalışmadaki amacım, Hitters veri seti için makine öğrenmesi modellerini kurup, hata skorlarını minimize etmektir. Bu amaçla yaptığım çalışmalar aşağıdadır :

#### **1).** Hitters Veri Seti okundu.
#### **2).** Keşifçi Veri Analizi ile;
* veri setinin yapısal bilgilerine bakıldı.
* veri setindeki değişkenlerin tiplerine bakıldı.
* object tipini kategorik tipe dönüştürme işlemi yapıldı.
* veri setinin boyut bilgisine erişildi.
* veri setindeki hangi değişkenden kaç tane eksik gözlem olduğu bilgisine erişildi. Sadece bağımlı değişkenimiz olan "Salary" de 59 adet eksik gözlem olduğu gözlendi.
* veri setinin betimsel istatistiklerine bakıldı.
* veri setindeki kategorik değişkenlere erişilip, bu değişkenlerin sınıfları görselleştirildi.
* veri setindeki sürekli değişkenlere erişilip, betimsel istatistiklerine bakıldı.

#### **3).** Veri Ön İşleme kısmında; 
Veri setindeki sürekli değişken ve kategorik değişkenlere ayrı ayrı işlemler yapıldı. Çünkü kategorik değişkenlere Label Encoding yöntemi uygulandıktan sonra, aykırı gözlem analizine kategorik değişkenler dahil edilmek istenmedi.
* sürekli değişkenlerde, hangi değişkenden kaç tane eksik gözlem olduğuna bakıldı.
* sadece "Salary" değişkeninde olan eksik gözlemler, KNNImputer yaklaşımı ile dolduruldu.
* kategorik değişkenlere Label Encoding dönüşümü uygulandı.
* sürekli değişkenlere aykırı gözlemler için LOF yöntemi ile çok değişkenli aykırı gözlem analizi yapıldı ve aykırı olan değerler veri setinden uzaklaştırıldı.
* sürekli değişkenlerden uzaklaştırılan indexlerin kategorik değişken dataframeinden de silinmesi sağlandı. 
* süreki ve kategorik değişken dataframelerinde ayrı ayrı index resetleme işlemi yapıldı.
* kategorik değişkeni içermeyen ve bağımlı değişkenin(Salary) atılmış hali üzerinden standardizasyon işlemi uygulandı.
* değişken standardizasyonundan sonra, model kurulmadan önce, normalize edilmiş sürekli değişken ve kategorik değişkenler birleştirilerek, veri setinin son halini alması sağlandı.

#### **4).** Model Kurma aşamasında;

Öğrendiğimiz makine öğrenmesi modelleri ile ilk önce herhangi bir hiperparametre optimizasyonu uygulanmadan, modellerin ilkel test hatası bulundu.
Öğrendiğimiz makine öğrenmesi modelleri ile kurulan modeller, hiperparametre optimizasyonu ile tune edilerek, bulunan test hatası düşürülmeye çalışılarak gerçek test hataları bulundu.

### 1). Kütüphane Import İşlemleri

In [ ]:
#!pip install xgboost
#!pip install lightgbm
#!pip install catboost

In [ ]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
import xgboost
from sklearn.impute import KNNImputer
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import LocalOutlierFactor
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, ElasticNet, Lasso, LassoCV
from sklearn.metrics import mean_squared_error

from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

### 2). Veriyi Okuma

In [ ]:
df = pd.read_csv("../input/hitters-baseball-data/Hitters.csv")
df_copy = df.copy()
df.head()

### 3). Keşifçi Veri Analizi

In [ ]:
#veri setinin yapısal bilgileri
df.info()

In [ ]:
#veri setindeki değişkenlerin bilgisi
df.dtypes

In [ ]:
#object tipini kategorik tipe dönüştürme işlemi
df.League = pd.Categorical(df.League)
df.Division = pd.Categorical(df.Division)
df.NewLeague = pd.Categorical(df.NewLeague)

In [ ]:
df.dtypes

In [ ]:
#veri setinde boyut bilgisi
df.shape

In [ ]:
#veri setinde hangi değişkenden kaç tane eksik gözlem var?
df.isnull().sum()

In [ ]:
#Betimsel İstatistikler
df.describe().T

### Veri setindeki kategorik değişkenlere erişim

In [ ]:
kat_df = df.select_dtypes(include=["category"])
kat_df.head()

In [ ]:
#League kategorik değişkeni için sınıflara, sınıf sayısına ve sınıf frekansına erişim ve görselleştirme
print(kat_df.League.unique())
print(kat_df["League"].value_counts().count())
print(kat_df["League"].value_counts())
print(df["League"].value_counts().plot.barh())

In [ ]:
#Division kategorik değişkeni için sınıflara, sınıf sayısına ve sınıf frekansına erişim ve görselleştirme
print(kat_df.Division.unique())
print(kat_df["Division"].value_counts().count())
print(kat_df["Division"].value_counts())
print(df["Division"].value_counts().plot.barh())

In [ ]:
#New League kategorik değişkeni için sınıflara,sınıf sayısına ve sınıf frekansına erişim ve görselleştirme
print(kat_df.NewLeague.unique())
print(kat_df["NewLeague"].value_counts().count())
print(kat_df["NewLeague"].value_counts())
print(df["NewLeague"].value_counts().plot.barh())

### Veri setindeki sürekli değişkenlere erişim

In [ ]:
num_df = df.select_dtypes(include = ["float64","int64"])
num_df

In [ ]:
#Betimsel İstatistikler
num_df.describe().T

### 4). Veri Ön İşleme

#### Eksik Değer

In [ ]:
num_df.isnull().sum()

In [ ]:
#Sadece "Salary" değişkeninde eksik değerler var. KNNImputer ile eksik değer dolduracağım
cols = num_df.columns
imputer = KNNImputer(n_neighbors=8)
num_df = imputer.fit_transform(num_df)
num_df=pd.DataFrame(num_df,columns=cols)
num_df.head

In [ ]:
num_df.isnull().sum()

In [ ]:
kat_df

#### LabelEncoder

In [ ]:
le = LabelEncoder()
kat_df['League'] = le.fit_transform(kat_df['League'])
kat_df['Division'] = le.fit_transform(kat_df['Division'])
kat_df['NewLeague'] = le.fit_transform(kat_df['NewLeague'])

In [ ]:
kat_df

#### Çok Değişkenli Aykırı Gözlem Analizi (LOF)

In [ ]:
clf = LocalOutlierFactor(n_neighbors = 20, contamination = 0.1)
clf.fit_predict(num_df)
df_scores = clf.negative_outlier_factor_
np.sort(df_scores)[0:30]


In [ ]:
sns.boxplot(df_scores);

In [ ]:
esik_deger = np.sort(df_scores)[7]
print(esik_deger)

In [ ]:
aykiri_tf = df_scores>esik_deger
aykirilar = num_df[~aykiri_tf]

In [ ]:
#aykirilar
aykirilar

In [ ]:
aykirilar.index

In [ ]:
num_df

In [ ]:
#aykiri olmayanlar
num_df = num_df[aykiri_tf]

# Kategorik değişkenlerden, indexi aykırı olanları uçuruyorum
kat_df = kat_df.drop(aykirilar.index)

In [ ]:
num_df

In [ ]:
kat_df

In [ ]:
#reset index işlemleri
num_df = num_df.reset_index(drop=True)
kat_df = kat_df.reset_index(drop=True)

In [ ]:
num_df

In [ ]:
kat_df

#### Değişken Standardizayonu

In [ ]:
y = num_df["Salary"]
X = num_df.drop('Salary', axis=1)
X

In [ ]:
scaled_cols=StandardScaler().fit_transform(X)
scaled_cols=pd.DataFrame(scaled_cols, columns=X.columns)

In [ ]:
#Değişken standardizasyonundan sonra, model kurmadan önce kategorik değişkenin birleştirilmesi
X = pd.concat([scaled_cols,kat_df], axis=1)
X

### 5). Model Kurma 

#### A).Doğrusal Regresyon Modelleri

In [ ]:
dogrusal_sonuc_ilkel = []
dogrusal_sonuc = []

In [ ]:
y

In [ ]:
X

In [ ]:
#y = num_df["Salary"]
#X = num_df.drop('Salary', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    random_state=46)


#### Doğrusal Regresyon

In [ ]:
linreg = LinearRegression()
linreg_model = linreg.fit(X_train,y_train)
linreg_y_pred = linreg_model.predict(X_test)
linreg_rmse = np.sqrt(mean_squared_error(y_test,linreg_y_pred))
dogrusal_sonuc_ilkel.append(("LinReg", linreg_rmse))
linreg_rmse

#### Ridge Regresyon

In [ ]:
ridreg = Ridge()
ridreg_model = ridreg.fit(X_train, y_train)
ridreg_y_pred = ridreg_model.predict(X_test)
ridreg_rmse = np.sqrt(mean_squared_error(y_test,ridreg_y_pred))
dogrusal_sonuc_ilkel.append(("RidgeReg", ridreg_rmse))
ridreg_rmse

In [ ]:
ridreg_model.alpha

#### Ridge Regresyon - HiperParametre Optimizasyonu

In [ ]:
alpha = [0,2,1]
ridreg_cv = RidgeCV(alphas = alpha, scoring = "neg_mean_squared_error", cv = 10, normalize = True)
ridreg_cv.fit(X_train, y_train)
print(ridreg_cv.alpha_)
#Final Model 
ridreg_tuned = Ridge(alpha = ridreg_cv.alpha_).fit(X_train,y_train)
ridreg_tuned_y_pred = ridreg_tuned.predict(X_test)
ridreg_tuned_rmse = np.sqrt(mean_squared_error(y_test,ridreg_tuned_y_pred))
print(ridreg_tuned_rmse)

dogrusal_sonuc.append(("RidgeReg",ridreg_cv.alpha_,ridreg_tuned_rmse))

#### Lasso Regresyon

In [ ]:
lasreg = Lasso()
lasreg_model = lasreg.fit(X_train,y_train)
lasreg_y_pred = lasreg_model.predict(X_test)
lasreg_rmse = np.sqrt(mean_squared_error(y_test,lasreg_y_pred))
dogrusal_sonuc_ilkel.append(("Lasso", lasreg_rmse))
lasreg_rmse

#### Lasso Regresyon - HiperParametre Optimizasyonu

In [ ]:
alpha = [0,5,1]
lasso_cv = LassoCV(alphas = alpha, cv = 10, normalize = True)
lasso_cv.fit(X_train, y_train)
print(lasso_cv.alpha_)
#Final Model 
lasso_tuned = Lasso(alpha = lasso_cv.alpha_).fit(X_train,y_train)
lasso_tuned_y_pred = lasso_tuned.predict(X_test)
lasso_tuned_rmse = np.sqrt(mean_squared_error(y_test,lasso_tuned_y_pred))
print(lasso_tuned_rmse)

dogrusal_sonuc.append(("Lasso",lasso_cv.alpha_,lasso_tuned_rmse))

#### ElasticNet Regresyon

In [ ]:
enet = ElasticNet()
enet_model = enet.fit(X_train,y_train)
enet_y_pred = enet_model.predict(X_test)
enet_rmse = np.sqrt(mean_squared_error(y_test,enet_y_pred))
dogrusal_sonuc_ilkel.append(("ENet", enet_rmse))
enet_rmse

#### ElasticNet Regresyon - HiperParametre Optimizasyonu

In [ ]:
enet_params = {"l1_ratio": [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1],
              "alpha":[0.1,0.01,0.001,0.2,0.3,0.5,0.8,0.9,1]}

enet_model = ElasticNet().fit(X_train,y_train)
enet_cv = GridSearchCV(enet_model, enet_params, cv = 10).fit(X, y)
print(enet_cv.best_params_)

#Final Model 
enet_tuned = ElasticNet(**enet_cv.best_params_).fit(X_train,y_train)
enet_tuned_y_pred = enet_tuned.predict(X_test)
enet_tuned_rmse = np.sqrt(mean_squared_error(y_test,enet_tuned_y_pred))
print(enet_tuned_rmse)

dogrusal_sonuc.append(("ENet",enet_cv.best_params_,enet_tuned_rmse))

#### Çıktı Hazırlama

In [ ]:
dogrusal_sonuc = pd.DataFrame(dogrusal_sonuc, columns = ["Model İsmi","Optimum Parametreler", "Test Hatası"])
dogrusal_sonuc = dogrusal_sonuc.set_index("Model İsmi")
print(dogrusal_sonuc)

dogrusal_sonuc_ilkel = pd.DataFrame(dogrusal_sonuc_ilkel,columns = ["Model İsmi","İlkel Test Hatası"])
dogrusal_sonuc_ilkel = dogrusal_sonuc_ilkel.set_index("Model İsmi")
print(dogrusal_sonuc_ilkel)

result_linear = pd.concat([dogrusal_sonuc_ilkel,dogrusal_sonuc,],axis = 1)
result_linear.sort_values("Test Hatası", ascending = True,inplace = True)
print(result_linear)




#### B).Doğrusal Olmayan Regresyon Modelleri

In [ ]:
models = []
sonuc_ilkel = []
sonuc = []

In [ ]:
#Model Nesneleri Oluştur
knn = KNeighborsRegressor().fit(X_train,y_train)
svr = SVR().fit(X_train,y_train)
ysa = MLPRegressor().fit(X_train,y_train)
cart = DecisionTreeRegressor().fit(X_train,y_train)
rf = RandomForestRegressor(random_state = 42).fit(X_train,y_train)
gbm = GradientBoostingRegressor().fit(X_train,y_train)
xgb = XGBRegressor().fit(X_train,y_train)
lgbm = LGBMRegressor().fit(X_train,y_train)
catb = CatBoostRegressor(verbose = False).fit(X_train,y_train)

#### Optimizasyonsuz

In [ ]:
models.append(('KNN', knn))
models.append(('SVR', svr))
models.append(('YSA',ysa))
models.append(('CART', cart))
models.append(('RF', rf))
models.append(('GBM', gbm))
models.append(("XGBoost", xgb))
models.append(("LightGBM", lgbm))
models.append(("CatBoost", catb))

In [ ]:
for name, modelobj in models:
        y_pred = modelobj.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        sonuc_ilkel.append((name,rmse))

In [ ]:
sonuc_ilkel_df = pd.DataFrame(sonuc_ilkel,columns = ["Model İsmi","İlkel Test Hatası"])
sonuc_ilkel_df = sonuc_ilkel_df.set_index("Model İsmi")
print(sonuc_ilkel_df)



## Model Tuning

In [ ]:
knn_params = {'n_neighbors': np.arange(1,10,1)}

svr_params = {"C": [0.01,0.001, 0.2, 0.1,0.5,0.8,0.9,1, 10, 100, 500,1000]}

mlp_params = {"alpha": [0.1, 0.01, 0.02, 0.001, 0.0001], 
             "hidden_layer_sizes": [(10,20), (5,5), (100,100), (1000,100,10)]}

cart_params = {"max_depth": [2,3,4,5,10,20, 100, 1000],
              "min_samples_split": [2,10,5,30,50,10]}

rf_params = {"max_depth": [5,8,10,None],
             "max_features": [2,5,10,15,17],
             "n_estimators": [100,200, 500, 1000],
             "min_samples_split": [2,5,10,20,30]}

gbm_params = {"learning_rate": [0.1,0.2,0.3],
             "max_depth": [2,3,4],
             "n_estimators": [100,200,300,400,500],
             "subsample": [0.5,1.0,2.0,3.0],
             "loss": ["ls","lad","quantile"]}

xgb_params = {"learning_rate": [0.1,0.01,0.5],
             "max_depth": [2,3,4,5,8],
             "n_estimators": [100,200,500,1000],
             "colsample_bytree": [0.4,0.7,1]}

lgbm_params = {"learning_rate": [0.1,0.2,0.3],
              "n_estimators": [100,200,300,400,500],
              "max_depth": [-1,0,2,4,5,7,10],
              "colsample_bytree": [1,0.8,0.5,0.4]}

catb_params = {"iterations": [200,500,100],
              "learning_rate": [0.01,0.1],
              "depth": [3,6,8]}

In [ ]:
#KNN
knn_cv_model = GridSearchCV(knn,knn_params,cv=10,n_jobs = -1).fit(X_train,y_train)
print(knn_cv_model.best_params_)

#Final Model 
knn_tuned = KNeighborsRegressor(**knn_cv_model.best_params_).fit(X_train,y_train)
knn_tuned_y_pred = knn_tuned.predict(X_test)
knn_tuned_rmse = np.sqrt(mean_squared_error(y_test,knn_tuned_y_pred))
print(knn_tuned_rmse)

sonuc.append(('KNN',knn_cv_model.best_params_,knn_tuned_rmse))



In [ ]:
#SVR
svr_cv_model = GridSearchCV(svr, svr_params, cv = 10, n_jobs = -1).fit(X_train, y_train)
print(svr_cv_model.best_params_)

#Final Model
svr_tuned = SVR(**svr_cv_model.best_params_).fit(X_train, y_train)
svr_tuned_y_pred = svr_tuned.predict(X_test)
svr_tuned_rmse = np.sqrt(mean_squared_error(y_test, svr_tuned_y_pred))
print(svr_tuned_rmse)

sonuc.append(('SVR',svr_cv_model.best_params_,svr_tuned_rmse))



In [ ]:
#YSA
mlp_cv_model = GridSearchCV(ysa, mlp_params, cv = 10, n_jobs = -1).fit(X_train, y_train)
print(mlp_cv_model.best_params_)

#Final Model
mlp_tuned = MLPRegressor(**mlp_cv_model.best_params_).fit(X_train,y_train)
mlp_tuned_y_pred = mlp_tuned.predict(X_test)
mlp_tuned_rmse = np.sqrt(mean_squared_error(y_test, mlp_tuned_y_pred))
print(mlp_tuned_rmse)

sonuc.append(('YSA',mlp_cv_model.best_params_,mlp_tuned_rmse))



In [ ]:
#CART
cart_cv_model = GridSearchCV(cart, cart_params, cv = 10, n_jobs = -1).fit(X_train, y_train)
print(cart_cv_model.best_params_)

#Final Model
cart_tuned = DecisionTreeRegressor(**cart_cv_model.best_params_).fit(X_train, y_train)
cart_tuned_y_pred = cart_tuned.predict(X_test)
cart_tuned_rmse = np.sqrt(mean_squared_error(y_test, cart_tuned_y_pred))
print(cart_tuned_rmse)

sonuc.append(('CART',cart_cv_model.best_params_,cart_tuned_rmse))


In [ ]:
#Random Forests
rf_cv_model = GridSearchCV(rf, rf_params, cv = 10, n_jobs = -1).fit(X_train, y_train)
print(rf_cv_model.best_params_)

#Final Model
rf_tuned = RandomForestRegressor(**rf_cv_model.best_params_).fit(X_train, y_train)
rf_tuned_y_pred = rf_tuned.predict(X_test)
rf_tuned_rmse = np.sqrt(mean_squared_error(y_test, rf_tuned_y_pred))
print(rf_tuned_rmse)

sonuc.append(('RF',rf_cv_model.best_params_,rf_tuned_rmse))

In [ ]:
#XGBoost
xgb_cv_model  = GridSearchCV(xgb,xgb_params, cv = 10, n_jobs = -1).fit(X_train, y_train)
print(xgb_cv_model.best_params_)

#Final Model
xgb_tuned = XGBRegressor(**xgb_cv_model.best_params_).fit(X_train,y_train)
xgb_tuned_y_pred = xgb_tuned.predict(X_test)
xgb_tuned_rmse = np.sqrt(mean_squared_error(y_test, xgb_tuned_y_pred))
print(xgb_tuned_rmse)

sonuc.append(('XGBoost',xgb_cv_model.best_params_,xgb_tuned_rmse))

In [ ]:
#LightGBM
lgbm_cv_model = GridSearchCV(lgbm,lgbm_params, cv = 10, n_jobs = -1).fit(X_train, y_train)
print(lgbm_cv_model.best_params_)

#Final Model
lgbm_tuned = LGBMRegressor(**lgbm_cv_model.best_params_).fit(X_train,y_train)
lgbm_tuned_y_pred = lgbm_tuned.predict(X_test)
lgbm_tuned_rmse = np.sqrt(mean_squared_error(y_test, lgbm_tuned_y_pred))
print(lgbm_tuned_rmse)

sonuc.append(('LightGBM',lgbm_cv_model.best_params_,lgbm_tuned_rmse))

In [ ]:
#CatBoost
catb_cv_model = GridSearchCV(catb,catb_params, cv = 10, n_jobs = -1).fit(X_train, y_train)
print(catb_cv_model.best_params_)

#Final Model
catb_tuned = CatBoostRegressor(**catb_cv_model.best_params_).fit(X_train,y_train)
catb_tuned_y_pred = catb_tuned.predict(X_test)
catb_tuned_rmse = np.sqrt(mean_squared_error(y_test, catb_tuned_y_pred))
print(catb_tuned_rmse)

sonuc.append(('CatBoost',catb_cv_model.best_params_,catb_tuned_rmse))


In [ ]:
#GBM
gbm_cv_model = GridSearchCV(gbm, gbm_params, cv = 10, n_jobs = -1).fit(X_train, y_train)
print(gbm_cv_model.best_params_)

#Final Model
gbm_tuned = GradientBoostingRegressor(**gbm_cv_model.best_params_).fit(X_train, y_train)
gbm_tuned_y_pred = gbm_tuned.predict(X_test)
gbm_tuned_rmse = np.sqrt(mean_squared_error(y_test, gbm_tuned_y_pred))
print(gbm_tuned_rmse)

sonuc.append(('GBM',gbm_cv_model.best_params_,gbm_tuned_rmse))

#### Çıktı Hazırlama

In [ ]:
sonuc_df = pd.DataFrame(sonuc, columns = ["Model İsmi","Optimum Parametreler", "Test Hatası"])
sonuc_df = sonuc_df.set_index("Model İsmi")
print(sonuc_df)

result_nonlinear = pd.concat([sonuc_ilkel_df,sonuc_df],axis = 1)
result_nonlinear.sort_values('Test Hatası', ascending=True, inplace = True)
print(result_nonlinear)

result = pd.concat([result_linear,result_nonlinear])
result.sort_values("Test Hatası", ascending = True, inplace = True)
print(result)

In [ ]:
result